# Image

In [ ]:
from aicsdeformation import AICSDeformation
import tifffile

cardio = tifffile.imread("data/piv.tiff")
cardio = AICSDeformation(cardio, t_index = 0)
cardio

# Processing: Grid Search

In [ ]:
from aicsdeformation import processing

grid = processing.grid_search_displacements((cardio[0], cardio[3]))
grid

# Deformation: Streamline Plots

In [ ]:
# Best displacement parameters found from grid search against ~50,000 parameter sets
# {'window_size': 40, 'overlap': 7, 'dt': 0.0045000000000000005, 'search_area_size': 40}
cardio.generate_displacements(
    window_size=40,
    overlap=7,
    dt=0.0045,
    search_area_size=26
)
cardio.displacements

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, axes = plt.subplots(11, int(len(cardio.displacements) / 11), figsize=(30, 20))
for i, ax in enumerate(axes.flatten()):
    ax.streamplot(
        cardio.displacements[i].x,
        cardio.displacements[i].y,
        cardio.displacements[i].u,
        cardio.displacements[i].v,
        density=[0.5, 1]
    )
    ax.set_title(f"From: {i}, To: {i + 1}")
    
plt.show()

# Deformation: Streamline Plot Overlaid On Animated Cardio

In [ ]:
from aicsdeformation import processing
import matplotlib
import matplotlib.animation
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML, Image
matplotlib.rc('animation', html='html5')

def animate_frames_and_streams(frames, window_size=40, overlap=7, dt=0.0045, search_area_size=40):
    # Create displacement
    entry = processing.calculate_displacement(
        frames,
        window_size=window_size,
        overlap=overlap,
        dt=dt,
        search_area_size=search_area_size
    )
    processed = processing.process_displacement(entry,
        image_size=frames[0].shape,
        window_size=window_size,
        overlap=overlap
    )
    
    fig = plt.figure(figsize=(10,10));
    ax = fig.add_axes([0,0,1,1])
    ax.axis('off');
    ax.streamplot(processed.x, processed.y, processed.u, processed.v, color='w')
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    image = ax.imshow(frames[0], extent=[xmin, xmax, ymin, ymax])

    def animate(i):
        image.set_data(frames[i%len(frames)])
        return (image, )

    anim = matplotlib.animation.FuncAnimation(fig,
        animate,
        frames=len(frames),
        interval=1000,
        blit=True
    )
    
    plt.close(fig)
    return anim

In [ ]:
HTML(animate_frames_and_streams((cardio[0], cardio[3])).to_html5_video())